In [1]:
import sqlalchemy as db
import pandas as pd

In [2]:
#create a sqlite database
engine=db.create_engine('sqlite:///database.db',echo=False)
md = db.MetaData()
connection = engine.connect()

# pull in the ecde cases data and dump to the database.
url = 'https://www.ecdc.europa.eu/en/cases-2019-ncov-eueea'
df = pd.read_html(url,header=0)[0]
sqlite_table = 'Test'
df.to_sql(sqlite_table, connection, if_exists='fail')

In [3]:
query = 'select * from TEST'
result = pd.read_sql(query,engine)
result[:5]

,index,Country,Cases,Deaths,Date of data collection
0,0,Austria,1211208,12612,09/12/2021
1,1,Belgium,1898403,27416,07/12/2021
2,2,Bulgaria,709537,29279,09/12/2021
3,3,Croatia,639778,11383,09/12/2021
4,4,Cyprus,138733,605,09/12/2021


In [4]:
tbl = db.Table('Test',md,autoload=True,autoload_with=engine)

print([i.name for i in tbl.columns])

pick = [tbl.c['Date of data collection']]

query= db.select([*pick,db.func.sum(tbl.c['Cases'])
                 ]). \
        group_by(*pick). \
        order_by(*pick)

result = pd.read_sql(query,engine)
result[:5]

['index', 'Country', 'Cases', 'Deaths', 'Date of data collection']


,Date of data collection,sum_1
0,07/12/2021,8368061
1,08/12/2021,1251457
2,09/12/2021,39451416


In [5]:
query= db.select([*pick,
                  db.case([(tbl.c['Country']=='Belgium',tbl.c['Cases'])]
                          ,else_=0).label('Belgium'),
                  db.case([(tbl.c['Country']=='Spain',tbl.c['Cases'])]
                          ,else_=0).label('Spain'),
                 ])

result = pd.read_sql(query,engine)
result[:5]

,Date of data collection,Belgium,Spain
0,09/12/2021,0,0
1,07/12/2021,1898403,0
2,09/12/2021,0,0
3,09/12/2021,0,0
4,09/12/2021,0,0


In [6]:
query= db.select([db.func.count(db.distinct(tbl.c['Country'])).label('Distinct_countries')
                 ])

result = pd.read_sql(query,engine)
result[:5]

,Distinct_countries
0,30
